In [1]:
# import cv2
import numpy as np
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt

In [2]:
interpreter = tf.lite.Interpreter(model_path = r'movenet.tflite')
interpreter.allocate_tensors()

Flipping Training Images

In [3]:
from PIL import Image
import os

def flip_images(input_dir, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Loop through all the files in the input directory
    for filename in os.listdir(input_dir):
        # Construct the full file path
        img_path = os.path.join(input_dir, filename)
        
        # Check if the file is an image
        if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            # Open the image
            img = Image.open(img_path)
            
            # Flip the image horizontally
            flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
            
            # Save the flipped image to the output directory
            flipped_img.save(os.path.join(output_dir, filename+'_flipped.jpg'))
            print(f"Flipped and saved: {filename}")

# Example usage:
input_directory = r'C:\Users\2001l\OneDrive\Desktop\squat\upPosition'
output_directory = r'C:\Users\2001l\OneDrive\Desktop\squat\upPosition'

flip_images(input_directory, output_directory)


Flipped and saved: 0_960x540.jpg
Flipped and saved: 0_960x540.jpg_flipped.jpg
Flipped and saved: 0_960x540.jpg_flipped.jpg_flipped.jpg
Flipped and saved: 2potEB8qNMABrHqjfWIpOOvbyAbsNmbLb7_SAM9jEQE.jpg
Flipped and saved: 2potEB8qNMABrHqjfWIpOOvbyAbsNmbLb7_SAM9jEQE.jpg_flipped.jpg
Flipped and saved: 2potEB8qNMABrHqjfWIpOOvbyAbsNmbLb7_SAM9jEQE.jpg_flipped.jpg_flipped.jpg
Flipped and saved: 362-1.jpg
Flipped and saved: 362-1.jpg_flipped.jpg
Flipped and saved: 362-1.jpg_flipped.jpg_flipped.jpg
Flipped and saved: 524.jpg
Flipped and saved: 524.jpg_flipped.jpg
Flipped and saved: 524.jpg_flipped.jpg_flipped.jpg
Flipped and saved: 6181ae52a3e0153fba71466fc28cb1fc595b4647_720.jpg
Flipped and saved: 6181ae52a3e0153fba71466fc28cb1fc595b4647_720.jpg_flipped.jpg
Flipped and saved: 6181ae52a3e0153fba71466fc28cb1fc595b4647_720.jpg_flipped.jpg_flipped.jpg
Flipped and saved: Barbell-Back-Squat.jpg
Flipped and saved: Barbell-Back-Squat.jpg_flipped.jpg
Flipped and saved: Barbell-Back-Squat.jpg_flipped.jp

In [4]:
from PIL import Image
import os

Dataframe of landmarks of the training Images

In [5]:
def landmarks_from_standing_images(input_dir,X,Y):
    for file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, file)
        
        # Check if the file is an image
        if os.path.isfile(img_path) and file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img = Image.open(img_path)
            img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 192, 192)
            input_image = tf.cast(img, dtype=tf.float32)

            # Setup input and output
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()

            #make predictions
            interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
            interpreter.invoke()
            keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
            landmarks = keypoints_with_scores

            X.append(landmarks)
            Y.append(0)
    return X,Y

def landmarks_from_squating_images(input_dir,X,Y):
    for file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, file)
        
        # Check if the file is an image
        if os.path.isfile(img_path) and file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img = Image.open(img_path)
            img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 192, 192)
            input_image = tf.cast(img, dtype=tf.float32)

            # Setup input and output
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()

            #make predictions
            interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
            interpreter.invoke()
            keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
            landmarks = keypoints_with_scores

            X.append(landmarks)
            Y.append(1)
    return X,Y
            

In [6]:
X,Y = landmarks_from_standing_images(r'upPosition',[],[])

In [7]:
len(Y)

52

In [8]:
X,Y = landmarks_from_squating_images(r'downPosition',X,Y)

In [9]:
import pandas as pd
# Generate column names
columns = [f"{i}_x" for i in range(1, 18)] + [f"{i}_y" for i in range(1, 18)] + ['Y']

# Create an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)

# Display the DataFrame to verify
print(df)

Empty DataFrame
Columns: [1_x, 2_x, 3_x, 4_x, 5_x, 6_x, 7_x, 8_x, 9_x, 10_x, 11_x, 12_x, 13_x, 14_x, 15_x, 16_x, 17_x, 1_y, 2_y, 3_y, 4_y, 5_y, 6_y, 7_y, 8_y, 9_y, 10_y, 11_y, 12_y, 13_y, 14_y, 15_y, 16_y, 17_y, Y]
Index: []

[0 rows x 35 columns]


In [10]:
df = pd.DataFrame(columns=columns)

y = 0
for entry in X:
    new_row = {col: None for col in columns}
    x = 0
    for x in range(1,18):
        xName = str(x) + '_x'
        yName = str(x) + '_y'
        new_row[xName] = entry[0][0][x-1][0]
        new_row[yName] = entry[0][0][x-1][1]
        x = x+1
    new_row['Y'] = Y[y]
    y = y+1
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\2001l\AppData\Local\Temp\ipykernel_56372\1965758734.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


In [11]:
df

,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,Y
0,0.332451,0.327286,0.328188,0.335584,0.337321,0.376591,0.380630,0.395435,0.405161,0.371132,...,0.455058,0.564492,0.452833,0.538660,0.492199,0.552743,0.490987,0.564235,0.489883,0
1,0.337427,0.330524,0.334669,0.338237,0.339605,0.375162,0.382447,0.390089,0.408159,0.346712,...,0.430101,0.523434,0.440300,0.505220,0.463085,0.507366,0.448752,0.511667,0.434354,0
2,0.248382,0.234584,0.236951,0.245007,0.244894,0.292224,0.277356,0.367411,0.338310,0.382827,...,0.713014,0.429093,0.720514,0.516982,0.610946,0.498965,0.643391,0.495000,0.678543,0
3,0.252336,0.240113,0.242199,0.246664,0.248041,0.277887,0.293914,0.341917,0.363804,0.387278,...,0.570867,0.278646,0.568406,0.385382,0.474913,0.346625,0.487613,0.319134,0.507263,0
4,0.241581,0.232052,0.231702,0.251969,0.244944,0.315338,0.316320,0.416093,0.418072,0.356919,...,0.445377,0.497518,0.468692,0.557133,0.482137,0.591745,0.482272,0.614701,0.470514,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.269335,0.255440,0.262633,0.248578,0.264382,0.315988,0.331319,0.370942,0.472705,0.351692,...,0.424774,0.252384,0.563788,0.183186,0.268853,0.527715,0.553091,0.381434,0.422276,1
92,0.152496,0.131350,0.130717,0.147583,0.143705,0.248949,0.246971,0.266599,0.254043,0.241453,...,0.304535,0.161116,0.155328,0.733582,0.631184,0.520522,0.423227,0.661199,0.525378,1
93,0.157098,0.132782,0.134116,0.142868,0.145697,0.240384,0.252563,0.251068,0.265584,0.243522,...,0.660548,0.841031,0.831572,0.380024,0.262156,0.575911,0.483287,0.470470,0.330142,1
94,0.291138,0.281100,0.280294,0.301488,0.296908,0.372551,0.364195,0.506963,0.483484,0.417078,...,0.579118,0.587216,0.574139,0.729981,0.651935,0.581281,0.571586,0.655428,0.641182,1


In [23]:
df.to_csv('squatTrainSet.csv', index = False)

PermissionError: [Errno 13] Permission denied: 'squatTrainSet.csv'

In [12]:
df.corr()

,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,Y
1_x,1.000000,0.998039,0.997844,0.995700,0.994926,0.952709,0.956751,0.583042,0.588549,0.567609,...,0.224039,-0.104012,0.121761,-0.067104,0.104824,-0.076732,0.138374,-0.164113,0.215662,0.341673
2_x,0.998039,1.000000,0.999667,0.997437,0.996860,0.943564,0.947252,0.572502,0.578259,0.575627,...,0.234923,-0.097371,0.129010,-0.079874,0.104086,-0.093967,0.155911,-0.176812,0.219017,0.300589
3_x,0.997844,0.999667,1.000000,0.996829,0.996853,0.941172,0.946170,0.569052,0.576690,0.576046,...,0.235410,-0.100278,0.129614,-0.084265,0.101632,-0.098475,0.159378,-0.182077,0.220013,0.298172
4_x,0.995700,0.997437,0.996829,1.000000,0.997696,0.954678,0.956737,0.602861,0.604431,0.590388,...,0.228198,-0.105833,0.119764,-0.064791,0.113489,-0.085685,0.141903,-0.166237,0.222908,0.310731
5_x,0.994926,0.996860,0.996853,0.997696,1.000000,0.951815,0.957361,0.592259,0.601820,0.584815,...,0.232072,-0.088387,0.136466,-0.095793,0.084356,-0.083430,0.145055,-0.188734,0.201952,0.309453
6_x,0.952709,0.943564,0.941172,0.954678,0.951815,1.000000,0.983856,0.751357,0.742581,0.634628,...,0.173009,-0.064274,0.116608,-0.036414,0.060884,0.060999,0.015900,-0.104399,0.133075,0.466240
7_x,0.956751,0.947252,0.946170,0.956737,0.957361,0.983856,1.000000,0.732863,0.748849,0.621902,...,0.156591,-0.090410,0.095773,-0.036250,0.064012,0.038773,0.003441,-0.104989,0.141819,0.472527
8_x,0.583042,0.572502,0.569052,0.602861,0.592259,0.751357,0.732863,1.000000,0.975522,0.735120,...,0.031899,-0.037885,0.044048,0.087686,0.048325,0.276466,-0.248980,0.104131,-0.010359,0.397643
9_x,0.588549,0.578259,0.576690,0.604431,0.601820,0.742581,0.748849,0.975522,1.000000,0.722057,...,0.024481,-0.033608,0.070537,-0.002592,-0.026877,0.264549,-0.233267,0.039490,-0.044752,0.403667
10_x,0.567609,0.575627,0.576046,0.590388,0.584815,0.634628,0.621902,0.735120,0.722057,1.000000,...,0.147299,0.010622,0.037156,-0.011528,0.027760,0.087406,-0.033764,-0.032226,0.074473,0.105376


In [13]:
x_cols = [col for col in columns if col != "Y"]
x_df = df[x_cols]
y_df = df[['Y']]

In [14]:
y_df['Y'] = y_df['Y'].astype('int')

C:\Users\2001l\AppData\Local\Temp\ipykernel_56372\2561977078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_df['Y'] = y_df['Y'].astype('int')


In [15]:
y_df.dtypes

Y    int32
dtype: object

In [16]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Define a simple Sequential model
model = Sequential([
    Dense(64, activation='relu', input_shape=(34,)),  # Adjust input_dim to match your data
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
              metrics=['accuracy'])

# Train the model
model.fit(x_df, y_df, epochs=150, validation_split=0.2)  # Adjust epochs as needed

# Save the model in a format suitable for TensorFlow Lite
model.save('model.h5')

c:\Users\2001l\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.6859 - loss: 0.6479 - val_accuracy: 0.0000e+00 - val_loss: 0.8980
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6507 - loss: 0.6494 - val_accuracy: 0.0000e+00 - val_loss: 0.9315
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6624 - loss: 0.6409 - val_accuracy: 0.0000e+00 - val_loss: 0.9704
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6937 - loss: 0.6209 - val_accuracy: 0.0000e+00 - val_loss: 0.9938
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6937 - loss: 0.6174 - val_accuracy: 0.0000e+00 - val_loss: 1.0145
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6741 - loss: 0.6236 - val_accuracy: 0.0000e+00 - val_loss: 1.0490
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6663 - loss: 0.6268 - val_accuracy: 0.0000e+00 - val_loss: 1.0880
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6702 - loss: 0.6222 - val

In [17]:
def landmarks_from_squating_images(input_dir,X,Y):
    for file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, file)
        
        # Check if the file is an image
        if os.path.isfile(img_path) and file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img = Image.open(img_path)
            img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 192, 192)
            input_image = tf.cast(img, dtype=tf.float32)

            # Setup input and output
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()

            #make predictions
            interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
            interpreter.invoke()
            keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
            landmarks = keypoints_with_scores

            X.append(landmarks)
            Y.append(1)
    return X,Y

def image_prediction(image_path):
    img = Image.open(image_path)
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    #make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    landmarks = keypoints_with_scores
    return landmarks[0][0]



In [20]:
test_X,test_Y = landmarks_from_squating_images('test images/sit',[],[])
test_X,test_Y = landmarks_from_standing_images('test images/stand',test_X,test_Y)

In [21]:
len(test_X)

10

In [22]:
test_columns = [cols for cols in columns if cols != 'Y']

df_test = pd.DataFrame(columns=test_columns)

for entry in test_X:
    new_row = {col: None for col in test_columns}
    x = 0
    for x in range(1,18):
        xName = str(x) + '_x'
        yName = str(x) + '_y'
        new_row[xName] = entry[0][0][x-1][0]
        new_row[yName] = entry[0][0][x-1][1]
        x = x+1
    df_test = pd.concat([df_test, pd.DataFrame([new_row])], ignore_index=True)
print(df_test)

        1_x       2_x       3_x       4_x       5_x       6_x       7_x  \
0  0.158883  0.129680  0.128846  0.141852  0.142761  0.247204  0.251513   
1  0.422304  0.404254  0.404367  0.414068  0.414410  0.490574  0.495341   
2  0.396914  0.384297  0.385278  0.400497  0.403513  0.484471  0.460770   
3  0.547260  0.536314  0.538534  0.546335  0.548772  0.611116  0.613522   
4  0.148019  0.126063  0.125789  0.158340  0.144929  0.266824  0.281123   
5  0.092995  0.069636  0.069766  0.071858  0.072698  0.131020  0.142498   
6  0.179094  0.169309  0.169256  0.192265  0.193769  0.293402  0.265297   
7  0.355024  0.343350  0.343809  0.354249  0.354406  0.420247  0.419390   
8  0.122018  0.101319  0.100482  0.117442  0.109174  0.195384  0.211573   
9  0.306403  0.289890  0.288576  0.280730  0.278254  0.337062  0.335010   

        8_x       9_x      10_x  ...       8_y       9_y      10_y      11_y  \
0  0.534532  0.501605  0.401277  ...  0.269241  0.339590  0.231762  0.254074   
1  0.625781  0

C:\Users\2001l\AppData\Local\Temp\ipykernel_56372\1551579132.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_test = pd.concat([df_test, pd.DataFrame([new_row])], ignore_index=True)


In [23]:
predictions = model.predict(df_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


In [24]:
predictions

array([[0.8971357 ],
       [0.88019645],
       [0.7610879 ],
       [0.95123905],
       [0.63445026],
       [0.02163909],
       [0.12390265],
       [0.44931784],
       [0.06793758],
       [0.11182853]], dtype=float32)